## Kaggle


In [1]:
# # !pip install kaggle
# from google.colab import files
# files.upload()

In [2]:
# ls -1ha kaggle.json

In [3]:
# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/

In [4]:
# !kaggle competitions download -c 6th-goorm-project-2-korean-mrc

In [5]:
# !unzip 6th-goorm-project-2-korean-mrc.zip

In [6]:
# !unzip dataset.zip

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## data

In [8]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## gogo

In [9]:
tokenizer_name = "monologg/koelectra-base-v3-finetuned-korquad"
model_name = "monologg/koelectra-base-v3-finetuned-korquad"

In [10]:
import json
import random

import torch
import numpy as np
from tqdm.notebook import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, AdamW

# **input data 추가**

In [11]:
with open("/content/drive/MyDrive/dataset/TL_span_extraction.json", 'rb') as f:
    input_dict_add = json.load(f)

In [12]:
input_dict_add['data'] = input_dict_add['data'][:4000]

In [13]:
input_dict_add['data'][0]

{'doc_id': '01601101.20210325201932006',
 'doc_title': '익산 서부권역 다목적 체육관, 다음달 개관',
 'doc_source': '전북일보',
 'doc_published': 20210325,
 'doc_class': {'class': '한국언론진흥재단 빅카인즈 뉴스기사', 'code': '스포츠'},
 'created': '20211217142431',
 'paragraphs': [{'context': '익산시 서부권역 다목적 체육관이 내달 개관을 앞두고 막바지 개관 준비가 한창이다. \n\n시에 따르면 시민들의 건강증진과 삶의 질 향상을 위한 생활체육 인프라 확충 등을 위해 지난 2019년 착공된 서부권역 다목적 체육관이 이달 말까지 시설물 준공을 완료하고 다음달부터 본격적인 개관 운영에 들어간다. \n\n전문 체육시설인 서부권역 다목적 체육관은 지하 1층과 지상 3층, 연면적 4719㎡ 규모로 총 사업비 124억5000만원이 투입됐다. \n\n수영장 6개 레인과 어린이풀장, 다목적실(에어로빅, 요가 등), 체력단련실, 어르신들을 위한 물리ㆍ재활 운동실 등이 들어서며, 수영, 헬스, 요가, 에어로빅, 아쿠아로빅 등 총 5가지 프로그램이 운영될 예정이다. \n\n첫 번째로 개관되는 시설은 헬스장이다. \n\n다음달 중순부터 회원 신청이 가능하며 체육관에 방문해 신청하면 된다. \n\n아울러 시는 코로나19 예방을 위해 소독작업은 물론 체육관 운영시간과 참여 인원을 축소해 운영하는 방안을 계획하고 있다. \n\n체육관 전체 소독은 하루 3회로 먼저 개관되는 헬스장의 경우 오후 1시부터 2시까지 1시간 동안 브레이크타임을 도입해 기구와 라커룸 등의 소독작업을 실시한다. \n\n시 관계자는 “시민들이 쾌적한 환경에서 다양한 체육활동에 참여할 수 있도록 시설 확충에 주력하고 있다”며“앞으로도 시민들의 생활체육활동을 지원할 수 있는 인프라를 지속적으로 확충해 나 가겠다”고 말했다.',
   'qas': [{'qa_type'

In [14]:
# len(input_dict_add['data'])
# # input_dict_add['data'][0]

# **추가 데이터 json 원본과 합치기**

In [15]:
# 추가 데이터셋 기존 데이터셋과 구조 같게 만들기

import re

for group in tqdm(input_dict_add['data']):       #딕셔너리 하나씩 꺼낸다.
        for passage in group['paragraphs']:     #딕셔너리의 paragraphs
            passage['context'] = re.sub(r'\n','',passage['context'])       #paragraphs의 context
            for qa in passage['qas']:           #paragraphs의 qas
                qa['question'] = qa['question'] + '?'      #paragraphs의 question
                # for answer in qa['answers']:    #question의 answers

                qa['answers']=[qa['answers']] # 기존 데이터셋 처럼 [] 안에 넣기


               

  0%|          | 0/4000 [00:00<?, ?it/s]

In [16]:
input_dict_add['data'][0]

{'doc_id': '01601101.20210325201932006',
 'doc_title': '익산 서부권역 다목적 체육관, 다음달 개관',
 'doc_source': '전북일보',
 'doc_published': 20210325,
 'doc_class': {'class': '한국언론진흥재단 빅카인즈 뉴스기사', 'code': '스포츠'},
 'created': '20211217142431',
 'paragraphs': [{'context': '익산시 서부권역 다목적 체육관이 내달 개관을 앞두고 막바지 개관 준비가 한창이다. 시에 따르면 시민들의 건강증진과 삶의 질 향상을 위한 생활체육 인프라 확충 등을 위해 지난 2019년 착공된 서부권역 다목적 체육관이 이달 말까지 시설물 준공을 완료하고 다음달부터 본격적인 개관 운영에 들어간다. 전문 체육시설인 서부권역 다목적 체육관은 지하 1층과 지상 3층, 연면적 4719㎡ 규모로 총 사업비 124억5000만원이 투입됐다. 수영장 6개 레인과 어린이풀장, 다목적실(에어로빅, 요가 등), 체력단련실, 어르신들을 위한 물리ㆍ재활 운동실 등이 들어서며, 수영, 헬스, 요가, 에어로빅, 아쿠아로빅 등 총 5가지 프로그램이 운영될 예정이다. 첫 번째로 개관되는 시설은 헬스장이다. 다음달 중순부터 회원 신청이 가능하며 체육관에 방문해 신청하면 된다. 아울러 시는 코로나19 예방을 위해 소독작업은 물론 체육관 운영시간과 참여 인원을 축소해 운영하는 방안을 계획하고 있다. 체육관 전체 소독은 하루 3회로 먼저 개관되는 헬스장의 경우 오후 1시부터 2시까지 1시간 동안 브레이크타임을 도입해 기구와 라커룸 등의 소독작업을 실시한다. 시 관계자는 “시민들이 쾌적한 환경에서 다양한 체육활동에 참여할 수 있도록 시설 확충에 주력하고 있다”며“앞으로도 시민들의 생활체육활동을 지원할 수 있는 인프라를 지속적으로 확충해 나 가겠다”고 말했다.',
   'qas': [{'qa_type': 1,
     'question_id': 4559297

In [17]:
with open("/content/drive/MyDrive/dataset/train.json", 'rb') as f:
    input_dict = json.load(f)
input_dict["data"][0]

{'title': '제주도 장마 시작 … 중부는 이달 말부터',
 'paragraphs': [{'context': '올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은 이달 말께 장마가 시작될 전망이다.17일 기상청에 따르면 제주도 남쪽 먼바다에 있는 장마전선의 영향으로 이날 제주도 산간 및 내륙지역에 호우주의보가 내려지면서 곳곳에 100㎜에 육박하는 많은 비가 내렸다. 제주의 장마는 평년보다 2~3일, 지난해보다는 하루 일찍 시작됐다. 장마는 고온다습한 북태평양 기단과 한랭 습윤한 오호츠크해 기단이 만나 형성되는 장마전선에서 내리는 비를 뜻한다.장마전선은 18일 제주도 먼 남쪽 해상으로 내려갔다가 20일께 다시 북상해 전남 남해안까지 영향을 줄 것으로 보인다. 이에 따라 20~21일 남부지방에도 예년보다 사흘 정도 장마가 일찍 찾아올 전망이다. 그러나 장마전선을 밀어올리는 북태평양 고기압 세력이 약해 서울 등 중부지방은 평년보다 사나흘가량 늦은 이달 말부터 장마가 시작될 것이라는 게 기상청의 설명이다. 장마전선은 이후 한 달가량 한반도 중남부를 오르내리며 곳곳에 비를 뿌릴 전망이다. 최근 30년간 평균치에 따르면 중부지방의 장마 시작일은 6월24~25일이었으며 장마기간은 32일, 강수일수는 17.2일이었다.기상청은 올해 장마기간의 평균 강수량이 350~400㎜로 평년과 비슷하거나 적을 것으로 내다봤다. 브라질 월드컵 한국과 러시아의 경기가 열리는 18일 오전 서울은 대체로 구름이 많이 끼지만 비는 오지 않을 것으로 예상돼 거리 응원에는 지장이 없을 전망이다.',
   'qas': [{'question': '북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?',
     'answers': [{'text': '한 달가량', 'answer_start': 478},
      {'text': '한 달', 'answer_start': 478}],
     'guid': '798db07f0b9046759deed9d4a

In [18]:
input_dict['data'][20:40]

[{'title': '존 윅',
  'paragraphs': [{'context': '《존 윅》(John Wick)은 데이비드 리치와 채드 스타헬스키가 연출한 2014년 공개된 미국의 네오 누아르 액션 스릴러 영화이다. 키아누 리브스, 미카엘 뉘크비스트, 알피 앨런, 에이드리언 팰리키, 브리짓 모이나한, 딘 윈터스, 이언 맥셰인, 존 레귀자모, 윌럼 더포등이 출연했다. 존 윅의 첫 시작 작품이며, 빈티지 자동차와 최근에 사망한 아내가 선물로 남긴 강아지를 죽인것에 대한 복수를 하고자 하는 은퇴한 청부 살인업자 존 윅 (리브스)의 대한 이야기를 다루고 있다. 리치와 스타헬스키는 영화를 함께 연출했지만, 리치는 크레딧에 오르지 않았다. \n\n2012년에 각본을 완료했었던 데릭 콜스테드가 각본을 썼고 선더 로드 픽쳐스를 통해 제작되었다. 선더 로드 픽쳐스 사의 배질 이와닉, 리치, 에바 롱고리아, 마이클 위더릴이 제작에 참여했다. 스타헬스키와 리치에게는 제2 제작진 감독과 스턴트맨으로서 경력을 시작한 이후 첫 감독으로서의 데뷔작이다. 그들은 매트릭스 트릴로지에서 스턴트맨으로서 리브스와 함께 작업한 바가 있다.',
    'qas': [{'question': '<존 윅>의 장르는?',
      'answers': [{'text': '네오 누아르 액션 스릴러', 'answer_start': 55}],
      'guid': 'fa379680fe04437caf4603358b901b2a'}]}],
  'news_category': None,
  'source': 'wikipedia'},
 {'title': '은미희',
  'paragraphs': [{'context': '1982년 방송통신대학 진학과 이듬해 광주문화방송 성우로서의 직업...하지만 그 어느것도 그녀에게 완전한 만족을 주지 못했고 1990년, 결국엔 모든 일을 접고 본격적으로 소설쓰기에 매달리게 된다. 은미희의 서른 살, 새롭게 인생을 시작하기로 결심한 그녀는 6년간의 습작기간을 거쳐 1996년 단편《누

In [19]:
# 기존 train 데이셋 \n 제거


import re

for group in tqdm(input_dict['data']):       #딕셔너리 하나씩 꺼낸다.
        for passage in group['paragraphs']:     #딕셔너리의 paragraphs
            passage['context'] = re.sub(r'\n','',passage['context'])       #paragraphs의 context
            # for qa in passage['qas']:           #paragraphs의 qas
                # qa['question'] = qa['question'] + '?'      #paragraphs의 question
                # for answer in qa['answers']:    #question의 answers

                # qa['answers']=[qa['answers']] # 기존 데이터셋 처럼 [] 안에 넣기


  0%|          | 0/9789 [00:00<?, ?it/s]

In [20]:
input_dict['data'][20:40]

[{'title': '존 윅',
  'paragraphs': [{'context': '《존 윅》(John Wick)은 데이비드 리치와 채드 스타헬스키가 연출한 2014년 공개된 미국의 네오 누아르 액션 스릴러 영화이다. 키아누 리브스, 미카엘 뉘크비스트, 알피 앨런, 에이드리언 팰리키, 브리짓 모이나한, 딘 윈터스, 이언 맥셰인, 존 레귀자모, 윌럼 더포등이 출연했다. 존 윅의 첫 시작 작품이며, 빈티지 자동차와 최근에 사망한 아내가 선물로 남긴 강아지를 죽인것에 대한 복수를 하고자 하는 은퇴한 청부 살인업자 존 윅 (리브스)의 대한 이야기를 다루고 있다. 리치와 스타헬스키는 영화를 함께 연출했지만, 리치는 크레딧에 오르지 않았다. 2012년에 각본을 완료했었던 데릭 콜스테드가 각본을 썼고 선더 로드 픽쳐스를 통해 제작되었다. 선더 로드 픽쳐스 사의 배질 이와닉, 리치, 에바 롱고리아, 마이클 위더릴이 제작에 참여했다. 스타헬스키와 리치에게는 제2 제작진 감독과 스턴트맨으로서 경력을 시작한 이후 첫 감독으로서의 데뷔작이다. 그들은 매트릭스 트릴로지에서 스턴트맨으로서 리브스와 함께 작업한 바가 있다.',
    'qas': [{'question': '<존 윅>의 장르는?',
      'answers': [{'text': '네오 누아르 액션 스릴러', 'answer_start': 55}],
      'guid': 'fa379680fe04437caf4603358b901b2a'}]}],
  'news_category': None,
  'source': 'wikipedia'},
 {'title': '은미희',
  'paragraphs': [{'context': '1982년 방송통신대학 진학과 이듬해 광주문화방송 성우로서의 직업...하지만 그 어느것도 그녀에게 완전한 만족을 주지 못했고 1990년, 결국엔 모든 일을 접고 본격적으로 소설쓰기에 매달리게 된다. 은미희의 서른 살, 새롭게 인생을 시작하기로 결심한 그녀는 6년간의 습작기간을 거쳐 1996년 단편《누에는 고

In [21]:
input_dict['data'] = input_dict['data'] + input_dict_add['data']

In [22]:
len(input_dict['data'])

13789

In [23]:
import json

with open('./new_train.json','w') as f:
  json.dump(input_dict, f, ensure_ascii=False, indent=4)

In [24]:
input_dict["data"][0]['paragraphs'][0]['qas'][0]

{'question': '북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?',
 'answers': [{'text': '한 달가량', 'answer_start': 478},
  {'text': '한 달', 'answer_start': 478}],
 'guid': '798db07f0b9046759deed9d4a35ce31e'}

In [25]:
# for i in input_dict["data"][0]['paragraphs'][0]['qas'][0]['answers']:
#   print(i)

In [26]:
# # 추가 데이터용 split_point()

# from copy import deepcopy
# def split_input_dict(input_dict, ratio = 0.1, seed = 42):
#     input_dict['data'] = input_dict['data'][:9000]
#     split_point = int(len(input_dict['data']) * ratio)
#     random.seed(seed)
#     random.shuffle(input_dict['data'])
#     valid_dict = deepcopy(input_dict)
#     train_dict = input_dict

#     valid_dict['data'] = input_dict['data'][:split_point]
#     train_dict['data'] = input_dict['data'][split_point:]
#     return train_dict, valid_dict

In [27]:
from copy import deepcopy
def split_input_dict(input_dict, ratio = 0.1, seed = 42):
    split_point = int(len(input_dict['data']) * ratio)
    random.seed(seed)
    random.shuffle(input_dict['data'])
    valid_dict = deepcopy(input_dict)
    train_dict = input_dict

    valid_dict['data'] = input_dict['data'][:split_point]
    train_dict['data'] = input_dict['data'][split_point:]
    return train_dict, valid_dict

In [28]:
# # 추가 데이터용 read_input()
# def read_input(path):
#     with open(path, 'rb') as f:
#         input_dict = json.load(f)

#     # 추가 데이터셋 기존 데이터셋과 구조 같게 만들기

#     for group in tqdm(input_dict['data']):       #딕셔너리 하나씩 꺼낸다.
#             for passage in group['paragraphs']:     #딕셔너리의 paragraphs
#                 context = passage['context']        #paragraphs의 context
#                 for qa in passage['qas']:           #paragraphs의 qas
#                     question = qa['question']       #paragraphs의 question
#                     # for answer in qa['answers']:    #question의 answers

#                     qa['answers']=[qa['answers']] # 기존 데이터셋 처럼 [] 안에 넣기

#     train_dict,valid_dict =split_input_dict(input_dict)

#     # 추가 데이터 구조 변경
    
#     train_contexts = []
#     train_questions = []
#     train_answers = []
#     for group in tqdm(train_dict['data']):       #딕셔너리 하나씩 꺼낸다.
#         for passage in group['paragraphs']:     #딕셔너리의 paragraphs
#             context = passage['context']        #paragraphs의 context
#             for qa in passage['qas']:           #paragraphs의 qas
#                 question = qa['question']       #paragraphs의 question
#                 for answer in qa['answers']:    #question의 answers
#                     ### context 넘겨서 자르기 ###
#                     target_context = context
#                     if answer['answer_start'] > 1400:
#                         target_context = target_context[1200:]
#                         answer['answer_start'] -= 1200 
#                     elif answer['answer_start'] > 1200:
#                         target_context = target_context[1000:]
#                         answer['answer_start'] -= 1000
#                     elif answer['answer_start'] > 1000:
#                         target_context = target_context[800:]
#                         answer['answer_start'] -= 800
#                     elif answer['answer_start'] > 800:
#                         target_context = target_context[600:]
#                         answer['answer_start'] -= 600
                    
#                     train_contexts.append(target_context)    #answers의 한 answer당 해당하는 context 저장
#                     ### context 넘겨서 자르기 ###
#                     train_questions.append(question)  #answers의 한 answer당 해당하는 question 저장
#                     train_answers.append(answer)      #answers의 한 answer 저장
  
#     valid_contexts = []
#     valid_questions = []
#     valid_answers = []
#     for group in tqdm(valid_dict['data']):       #딕셔너리 하나씩 꺼낸다.
#         for passage in group['paragraphs']:     #딕셔너리의 paragraphs
#             context = passage['context']        #paragraphs의 context
#             for qa in passage['qas']:           #paragraphs의 qas
#                 question = qa['question']       #paragraphs의 question
#                 for answer in qa['answers']:    #question의 answers
#                     ### context 넘겨서 자르기 ###
#                     target_context = context
#                     if answer['answer_start'] > 1400:
#                         target_context = target_context[1200:]
#                         answer['answer_start'] -= 1200 
#                     elif answer['answer_start'] > 1200:
#                         target_context = target_context[1000:]
#                         answer['answer_start'] -= 1000
#                     elif answer['answer_start'] > 1000:
#                         target_context = target_context[800:]
#                         answer['answer_start'] -= 800
#                     elif answer['answer_start'] > 800:
#                         target_context = target_context[600:]
#                         answer['answer_start'] -= 600
                    
#                     valid_contexts.append(target_context)    #answers의 한 answer당 해당하는 context 저장
#                     ### context 넘겨서 자르기 ###
#                     valid_questions.append(question)  #answers의 한 answer당 해당하는 question 저장
#                     valid_answers.append(answer)      #answers의 한 answer 저장

#     return train_contexts, train_questions, train_answers, valid_contexts, valid_questions, valid_answers


In [29]:
def read_input(path):
    with open(path, 'rb') as f:
        input_dict = json.load(f)
    train_dict,valid_dict =split_input_dict(input_dict)
    train_contexts = []
    train_questions = []
    train_answers = []
    for group in tqdm(train_dict['data']):       #딕셔너리 하나씩 꺼낸다.
        for passage in group['paragraphs']:     #딕셔너리의 paragraphs
            context = passage['context']        #paragraphs의 context
            for qa in passage['qas']:           #paragraphs의 qas
                question = qa['question']       #paragraphs의 question
                for answer in qa['answers']:    #question의 answers
                    ### context 넘겨서 자르기 ###
                    target_context = context
                    if answer['answer_start'] > 1400:
                        target_context = target_context[1200:]
                        answer['answer_start'] -= 1200 
                    elif answer['answer_start'] > 1200:
                        target_context = target_context[1000:]
                        answer['answer_start'] -= 1000
                    elif answer['answer_start'] > 1000:
                        target_context = target_context[800:]
                        answer['answer_start'] -= 800
                    elif answer['answer_start'] > 800:
                        target_context = target_context[600:]
                        answer['answer_start'] -= 600
                    
                    train_contexts.append(target_context)    #answers의 한 answer당 해당하는 context 저장
                    ### context 넘겨서 자르기 ###
                    train_questions.append(question)  #answers의 한 answer당 해당하는 question 저장
                    train_answers.append(answer)      #answers의 한 answer 저장
  
    valid_contexts = []
    valid_questions = []
    valid_answers = []
    for group in tqdm(valid_dict['data']):       #딕셔너리 하나씩 꺼낸다.
        for passage in group['paragraphs']:     #딕셔너리의 paragraphs
            context = passage['context']        #paragraphs의 context
            for qa in passage['qas']:           #paragraphs의 qas
                question = qa['question']       #paragraphs의 question
                for answer in qa['answers']:    #question의 answers
                    ### context 넘겨서 자르기 ###
                    target_context = context
                    if answer['answer_start'] > 1400:
                        target_context = target_context[1200:]
                        answer['answer_start'] -= 1200 
                    elif answer['answer_start'] > 1200:
                        target_context = target_context[1000:]
                        answer['answer_start'] -= 1000
                    elif answer['answer_start'] > 1000:
                        target_context = target_context[800:]
                        answer['answer_start'] -= 800
                    elif answer['answer_start'] > 800:
                        target_context = target_context[600:]
                        answer['answer_start'] -= 600
                    
                    valid_contexts.append(target_context)    #answers의 한 answer당 해당하는 context 저장
                    ### context 넘겨서 자르기 ###
                    valid_questions.append(question)  #answers의 한 answer당 해당하는 question 저장
                    valid_answers.append(answer)      #answers의 한 answer 저장

    return train_contexts, train_questions, train_answers, valid_contexts, valid_questions, valid_answers

In [30]:
# 추가 데이터셋용 add_end_idx
def add_end_idx(answers, contexts):
    for answer, context in zip(answers, contexts):
        gold_text = answer['text']
        start_idx = answer['answer_start']
        end_idx = start_idx + len(gold_text)
        
        answer['answer_end'] = end_idx # 추가

        if context[start_idx:end_idx] == gold_text:
            answer['answer_end'] = end_idx
        elif context[start_idx-1:end_idx-1] == gold_text:
            print("there is an unitended error in dataset") #이렇게까지 할 필요가 있나?
            answer['answer_start'] = start_idx - 1
            answer['answer_end'] = end_idx - 1
        elif context[start_idx-2:end_idx-2] == gold_text:
            print("there is an unitended error in dataset")
            answer['answer_start'] = start_idx - 2
            answer['answer_end'] = end_idx - 2

In [31]:
# def add_end_idx(answers, contexts):
#     for answer, context in zip(answers, contexts):
#         gold_text = answer['text']
#         start_idx = answer['answer_start']
#         end_idx = start_idx + len(gold_text)
        
#         if context[start_idx:end_idx] == gold_text:
#             answer['answer_end'] = end_idx
#         elif context[start_idx-1:end_idx-1] == gold_text:
#             print("there is an unitended error in dataset") #이렇게까지 할 필요가 있나?
#             answer['answer_start'] = start_idx - 1
#             answer['answer_end'] = end_idx - 1
#         elif context[start_idx-2:end_idx-2] == gold_text:
#             print("there is an unitended error in dataset")
#             answer['answer_start'] = start_idx - 2
#             answer['answer_end'] = end_idx - 2

In [32]:
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

In [33]:
question = "북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?"
context = "올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은 이달 말께 장마가 시작될 전망이다.17일 기상청에 따르면 제주도 남쪽 먼바다에 있는 장마전선의 영향으로 이날 제주도 산간 및 내륙지역에 호우주의보가 내려지면서 곳곳에 100㎜에 육박하는 많은 비가 내렸다. 제주의 장마는 평년보다 2~3일, 지난해보다는 하루 일찍 시작됐다. 장마는 고온다습한 북태평양 기단과 한랭 습윤한 오호츠크해 기단이 만나 형성되는 장마전선에서 내리는 비를 뜻한다.장마전선은 18일 제주도 먼 남쪽 해상으로 내려갔다가 20일께 다시 북상해 전남 남해안까지 영향을 줄 것으로 보인다. 이에 따라 20~21일 남부지방에도 예년보다 사흘 정도 장마가 일찍 찾아올 전망이다. 그러나 장마전선을 밀어올리는 북태평양 고기압 세력이 약해 서울 등 중부지방은 평년보다 사나흘가량 늦은 이달 말부터 장마가 시작될 것이라는 게 기상청의 설명이다. 장마전선은 이후 한 달가량 한반도 중남부를 오르내리며 곳곳에 비를 뿌릴 전망이다. 최근 30년간 평균치에 따르면 중부지방의 장마 시작일은 6월24~25일이었으며 장마기간은 32일, 강수일수는 17.2일이었다.기상청은 올해 장마기간의 평균 강수량이 350~400㎜로 평년과 비슷하거나 적을 것으로 내다봤다. 브라질 월드컵 한국과 러시아의 경기가 열리는 18일 오전 서울은 대체로 구름이 많이 끼지만 비는 오지 않을 것으로 예상돼 거리 응원에는 지장이 없을 전망이다."
tokenizer(context, question)

{'input_ids': [2, 3161, 27913, 14323, 4070, 6478, 4366, 9268, 4073, 4129, 6273, 5739, 4176, 18, 6265, 2446, 11574, 21397, 4112, 16071, 4275, 4176, 15632, 5283, 6282, 2336, 4112, 8003, 2633, 5253, 14323, 4070, 6273, 4587, 6585, 24387, 18, 6478, 4366, 13360, 4073, 6384, 4181, 9268, 9778, 2655, 13228, 4073, 3249, 4034, 14323, 4068, 4207, 4234, 6542, 10749, 6382, 9268, 24510, 2728, 14833, 23527, 4073, 24589, 6448, 4275, 4070, 25443, 4181, 4129, 8609, 4073, 6399, 5939, 4073, 13342, 4279, 4034, 2631, 4112, 2818, 4070, 8365, 4176, 18, 7068, 4234, 14323, 4034, 19835, 4275, 4176, 22, 98, 23, 4366, 16, 6346, 4275, 6913, 6674, 8879, 6273, 5739, 4176, 18, 14323, 4034, 22721, 4176, 4576, 4283, 2800, 20435, 27276, 4047, 3757, 5042, 3005, 4666, 4283, 30813, 29158, 4151, 27276, 4007, 6654, 7425, 4479, 4034, 14323, 4068, 4207, 4073, 4129, 7503, 4034, 2818, 4110, 2513, 7796, 18, 14323, 4068, 4207, 4112, 6363, 4366, 9268, 2655, 9778, 9757, 10749, 16021, 7483, 6224, 4366, 5253, 6304, 26635, 4151, 7943, 22

In [34]:
class KlueDataset(Dataset):
    def __init__(self, contexts, questions, answers, model_max_position_embedings, tokenizer):
        self.tokenizer = tokenizer
        self.answers = answers
        self.questions = questions
        self.contexts = contexts
        self.model_max_position_embedings = model_max_position_embedings
        print("Tokenizing ...")
        self.encodings = self.tokenizer(self.contexts, 
                                        self.questions,
                                        max_length=512, #512 truncation // 
                                        truncation=True,
                                        padding="max_length",
                                        return_token_type_ids=False)
        print("Done !!!")
        self.add_token_positions()
        
    def add_token_positions(self):
        start_positions = []
        end_positions = []
        for i in range(len(self.answers)):
            start_positions.append(self.encodings.char_to_token(i, self.answers[i]['answer_start']))
            end_positions.append(self.encodings.char_to_token(i, self.answers[i]['answer_end'] - 1)) # -1으로 : 진짜로 답이 있는 end_position 의 인덱스를 구함.(char_to_token은 인덱스를 구함)
            #https://huggingface.co/docs/tokenizers/v0.13.2/en/api/encoding#tokenizers.Encoding.char_to_token

            # positions 값이 None 값이라면, answer가 포함된 context가 잘렸다는 의미
            if start_positions[-1] is None:
                print("there is an error 1")
                start_positions[-1] = self.model_max_position_embedings
            if end_positions[-1] is None:
                print("there is an error 2")
                end_positions[-1] = self.model_max_position_embedings

        self.encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

        
    def get_data(self):
        return {"contexts":self.contexts, 'questions':self.questions, 'answers':self.answers}
    
    
    def get_encodings(self):
        return self.encodings
        
    
    def __getitem__(self, idx):
        return {key:torch.tensor(val[idx]) for key, val in self.encodings.items()}
    
    def __len__(self):
        return len(self.encodings['input_ids'])

In [35]:
train_contexts, train_questions, train_answers, valid_contexts, valid_questions, valid_answers = read_input("new_train.json")
add_end_idx(train_answers, train_contexts)                                                      #anwer 마다 answer_end 달아준다.
train_dataset = KlueDataset(train_contexts, train_questions, train_answers, 512, tokenizer)

add_end_idx(valid_answers, valid_contexts)                                                      #anwer 마다 answer_end 달아준다.
valid_dataset = KlueDataset(valid_contexts, valid_questions, valid_answers, 512, tokenizer)

  0%|          | 0/12411 [00:00<?, ?it/s]

  0%|          | 0/1378 [00:00<?, ?it/s]

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
there is an unitended error in dataset
there is an unitended error in dataset
there is an unitended error in dataset
there is an unitended error in dataset
there is an unitended error in dataset
there is an unitended error in dataset
there is an unitended error in dataset
there is an unitended error in dataset
there is an unitended error in dataset
there is an unitended error in dataset
there is an unitended error in dataset
there is an unitended error in dataset
there is an unitended error in dataset
there is an unitended error in dataset
there is an unitended error in dataset
there is an unitended error in dataset
there is an unitended error in dataset
there is an unitended error in dataset
there is an unitended error in dataset
there is an unitended error in dataset
there is an unitended error in dataset
there is an unitended error in dataset
there is an unitended error in dataset
there is an unitended error in dataset
there is an unitended error 

In [36]:
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

In [37]:
# !pip install wandb -qqq
# import wandb
# wandb.login()

In [38]:
# sweep_config = {
#       'name' : 'project2_TEST1.ipynb',
#       'method' : 'grid',
#       'metric':{
#           'name': 'total_valid_loss',
#           'goal': 'minimize'  
#       },
#       'parameters' : {
#           'learning_rate' : {
#               'values' : [1e-4, 2.5e-5, 5e-5, 7.5e-5,1e-5]  
#           },   
#           'batch_size' :{
#               'values' : [4,8,16]
#           },
#           'epochs' : {
#               'values' : [2] 
#           }
#       }
# }

# sweep_id = wandb.sweep(sweep_config)

In [39]:
EPOCH = 1
LEARNING_RATE = 5e-5
BATCH_SIZE = 16

In [40]:
# Levenshtein_distance (Evaluation)

import numpy 
import torch
import os

def levenshtein_distance(s1,s2, debug=False): #레벤슈타인 거리 eval / 정답 s1과 도출한 모델 s2 비교 평가
    if len(s1) < len(s2):
        return levenshtein_distance(s2, s1, debug)

    if len(s2) == 0:
        return len(s1)

    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))

        if debug:
            print(current_row[1:])

        previous_row = current_row

    return previous_row[-1] # levenshtein_distance 값 출력력


def LD_SCORE(start_positions, end_positions, input_ids, STA_logits, END_logits):

    answer1 = [] # 정답 저장
    answer2 = [] # 예측 정답 저장

    if len(input_ids) != BATCH_SIZE: #오류 해결
       print("input_ids ERROR")
       return 0

    for i in range(BATCH_SIZE): # 기존 정답 // index 1 is out of bounds for dimension 0 with size 1 오류 발생
        if input_ids[i] == [] :
            print("input_ids ERROR")
            continue
        else:
            PRED_IDE = input_ids[i][start_positions[i]: end_positions[i] + 1]
            PRED_ANS = tokenizer.decode(PRED_IDE)
            answer1.append(PRED_ANS)
            #print(PRED_ANS)

    for i in range(BATCH_SIZE): # 예측 정답
        if input_ids[i] == [] :
            print("input_ids ERROR")
            continue
        else:
            TK_start_index, TK_end_index = STA_logits.argmax(dim=-1), END_logits.argmax(dim=-1)
            PRED_IDE2 = input_ids[i][TK_start_index[i]: TK_end_index[i] + 1]
            PRED_ANS2 = tokenizer.decode(PRED_IDE2)
            answer2.append(PRED_ANS2)
            # print(PRED_ANS2)

    batch_score = LD_comparison(answer1, answer2)

    return batch_score


def LD_comparison(answer1,answer2):

    # 배치마다 레벤슈타인 거리 평균 구해서 출력. (train, valid 과정에서 배치마다 평균거리 구하고.)
    # train 전체 평균 거리
    # valid 전체 평균 거리

    batch_LD_score = []

    for i in range(BATCH_SIZE):
        if answer1[i] == answer2[i]: # 같으면 LD 구하는 과정 생략.
            batch_LD_score.append(0)
        else:
            batch_LD_score.append(levenshtein_distance(answer1[i],answer2[i]))

    sum_LD_score = sum(batch_LD_score)
    LD_avg = sum_LD_score / BATCH_SIZE # 배치의 레벤슈타인 거리 평균
    print(LD_avg)

    return LD_avg



'''
levenshtein_distance 값이 튀는 현상 방지.
 - 길이가 너무 긴 정답 삭제 max_length = 20
 - 정답과 어느정도 길이차이나는 값 삭제(위와 동일) 5.9 -> 2정도 차이나는 값 삭제 7.9
 

자연어처리
자연어처리과정 2
0 5

정답 길이보다 예측 정답 길이가 길지만 2배이상 차이가 나지 않는 경우에
0으로 처리하면 LD_SCORE가 더 안나옴
따라서 위 상황에서는 1.5 ~ 1.8 정도 해당하는 길이로 잘라서 반환.

test 답 X
train 과정 나온 답 -> 바꿔서 바꾼 데이터로 


** max_len = 5.9 * 2 = 12

train 2번
1 train 원래 데이터.
2 train LD 변환한 데이터로 한번.

'''



'\nlevenshtein_distance 값이 튀는 현상 방지.\n - 길이가 너무 긴 정답 삭제 max_length = 20\n - 정답과 어느정도 길이차이나는 값 삭제(위와 동일) 5.9 -> 2정도 차이나는 값 삭제 7.9\n \n\n자연어처리\n자연어처리과정 2\n0 5\n\n정답 길이보다 예측 정답 길이가 길지만 2배이상 차이가 나지 않는 경우에\n0으로 처리하면 LD_SCORE가 더 안나옴\n따라서 위 상황에서는 1.5 ~ 1.8 정도 해당하는 길이로 잘라서 반환.\n\ntest 답 X\ntrain 과정 나온 답 -> 바꿔서 바꾼 데이터로 \n\n\n** max_len = 5.9 * 2 = 12\n\ntrain 2번\n1 train 원래 데이터.\n2 train LD 변환한 데이터로 한번.\n\n'

In [41]:
# LD SCORE 저장
train_LD_avg = [] 
valid_LD_avg = []

def train_runner(model, train_dataset, valid_dataset , batch_size, num_train_epochs, learning_rate):

    #wandb.init(project = 'project2_test1',reinit=True)

    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    
    model.to(device)
    model.train()
    train_dataloader = DataLoader(dataset=train_dataset, batch_size=batch_size)
    valid_dataloader = DataLoader(dataset = valid_dataset, batch_size = batch_size)

    lowest_total_valid_loss = 9999.

    global_total_step = len(train_dataloader) * num_train_epochs
    optimizer = AdamW(model.parameters(), lr=learning_rate, weight_decay=0)
    print("TRAIN START")
    with tqdm(total=global_total_step, unit='step') as t:
        total = 0
        total_loss = 0
        for epoch in range(num_train_epochs):
            for iteration,batch in enumerate(train_dataloader):
                optimizer.zero_grad()
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                start_positions = batch['start_positions'].to(device)
                end_positions = batch['end_positions'].to(device)                

                outputs = model(input_ids,
                             attention_mask=attention_mask,
                             start_positions=start_positions,
                             end_positions=end_positions)
                
                ### LD_SCORE - train
                STA_logits, END_logits = outputs.start_logits, outputs.end_logits
                score_save1 = LD_SCORE(start_positions, end_positions, input_ids, STA_logits, END_logits)
                train_LD_avg.append(score_save1)
                #wandb.log({'train_batch_LD':score_save1})

                ####
                #loss_logit_change(start_positions, end_positions, input_ids, STA_logits, END_logits)
                loss = outputs.loss
                #wandb loss
                #wandb.log({'Train_loss':loss.item()})

                loss.backward()
                optimizer.step()
                
                batch_loss = loss.item() * len(input_ids)
                total += len(input_ids)
                total_loss += batch_loss
                global_total_step += 1
                t.set_postfix(loss="{:.6f}".format(total_loss / total), batch_loss="{:.6f}".format(batch_loss))
                t.update(1)

                # wandb loss
                #wandb.log({'Train_batch_loss':batch_loss})
                #wandb.log({'LOSS':total_loss / total})
                
                del input_ids
                del attention_mask
                del start_positions
                del end_positions
                del outputs
                del loss

                ## validation ##
                if iteration != 0 and iteration % int(len(train_dataloader) / 5) == 0:
                    total_valid_loss = 0
                    for batch_val in valid_dataloader:
                        model.eval()
                        optimizer.zero_grad()

                        input_ids = batch_val['input_ids'].to(device)
                        attention_mask = batch_val['attention_mask'].to(device)
                        start_positions = batch_val['start_positions'].to(device)
                        end_positions = batch_val['end_positions'].to(device)
                
                        with torch.no_grad():
                            outputs = model(input_ids,
                                    attention_mask=attention_mask,
                                    start_positions=start_positions,
                                    end_positions=end_positions)
                            
                            ### LD_SCORE - valid
                            STA_logits, END_logits = outputs.start_logits, outputs.end_logits
                            score_save2 = LD_SCORE(start_positions, end_positions, input_ids, STA_logits, END_logits)
                            valid_LD_avg.append(score_save2)

                            # wandb LD
                            #wandb.log({'Valid_batch_LD':score_save2})

                            loss = outputs.loss
                            total_valid_loss += loss.item()

                            # wandb loss
                            #wandb.log({'Valid_loss':loss.item()})
                            #wandb.log({'total_valid_loss':total_valid_loss})
                    
                    if total_valid_loss < lowest_total_valid_loss:
                        print(f"lowest_total_valid_loss: {total_valid_loss} epoch : {epoch} iteration : {iteration}")
                        torch.save(model.state_dict(),'./output_model_best')
                        lowest_total_valid_loss = total_valid_loss
                ## validation ##

    #model.save_pretrained("./klue_output_model")
    # train, valid 평균 LD 거리
    train_AVG = sum(train_LD_avg) / len(train_LD_avg)
    valid_AVG = sum(valid_LD_avg) / len(valid_LD_avg)
    #wandb.log({'train_AVG_LD':train_AVG})
    #wandb.log({'valid_AVG_LD':valid_AVG})

    print('train LD average',train_AVG,'valid LD average',valid_AVG)
    print("TRAIN END")

In [42]:
# wandb sweep train 실행.

# wandb.agent( sweep_id , function=train_runner, count=1)

In [43]:
train_runner(model,train_dataset,valid_dataset, BATCH_SIZE, EPOCH, LEARNING_RATE)

TRAIN START


/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/1447 [00:00<?, ?step/s]

146.4375
106.3125
76.5625
7.0
127.75
177.25
28.5
7.125
52.1875
63.125
18.5
128.6875
23.6875
8.5
222.625
41.875
19.5625
8.625
4.6875
75.9375
29.75
52.3125
300.3125
12.375
22.375
26.0625
38.9375
212.375
7.1875
6.75
1.875
113.75
4.9375
151.9375
117.75
71.1875
4.8125
7.0
48.3125
216.75
3.3125
68.5
75.5
6.5625
5.125
113.875
30.5
24.0
122.3125
4.25
143.1875
41.625
14.3125
25.625
34.25
88.0
209.25
65.25
80.1875
85.8125
8.8125
163.9375
103.875
23.125
44.4375
230.1875
14.875
4.625
46.125
6.75
212.25
3.375
2.625
3.0625
11.5625
45.125
193.875
32.875
28.125
11.375
29.375
20.6875
13.0625
3.1875
6.8125
14.1875
28.1875
4.8125
46.125
4.3125
2.5625
38.625
6.875
18.5
52.25
51.8125
65.9375
114.5
36.25
22.0
26.25
4.25
10.5
7.9375
11.125
218.3125
26.0625
25.8125
24.625
13.0
93.875
19.125
29.875
12.8125
360.375
180.125
19.4375
7.375
1.75
3.75
30.875
81.6875
17.0625
3.3125
11.5
11.125
124.6875
3.5625
246.625
16.0
5.0625
9.375
313.25
7.75
10.0625
126.5625
58.5
78.625
206.75
9.5625
18.9375
148.875
290.5
225.62

KeyboardInterrupt: ignored

In [44]:
MAX_LEN = 10

# train / valid 과정에서 나온 12 이상값 제거 / TEST 도 12이상 값 제거하니까

train_LD_avg = [v for v in train_LD_avg if v < MAX_LEN]  
TR_AVG = sum(train_LD_avg) / len(train_LD_avg) 

valid_LD_avg = [v for v in valid_LD_avg if v < MAX_LEN]
VA_AVG = sum(valid_LD_avg) / len(valid_LD_avg)

print('predicted Train Avgrage LD',TR_AVG,'predicted Valid Avgrage LD',VA_AVG) # 예측값 확인.

predicted Train Avgrage LD 4.740625 predicted Valid Avgrage LD 4.514030612244898


In [45]:
# \n 삭제 버전

def read_dev_klue(path):
    with open(path, 'rb') as f:
        klue_dict = json.load(f)

    contexts = []
    questions = []
    answers = []
    guids = []

    for group in tqdm(klue_dict['data']):
        for passage in group['paragraphs']:
            passage['context'] = re.sub(r'\n','',passage['context']) # \n 삭제 추가
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                guid = qa['guid']
                #temp_answer = []
                #for answer in qa['answers']:
                    #temp_answer.append(answer['text'])
                #if len(temp_answer) != 0: # answers의 길이가 0 == 답변할 수 없는 질문
                    #contexts.append(context)
                    #questions.append(question)
                    #answers.append(temp_answer)
                contexts.append(context)##
                questions.append(question)##
                guids.append(guid)

    #return contexts, questions, answers
    return contexts, questions , guids

In [46]:
#dev_contexts, dev_questions, dev_answers = read_dev_klue("test.json")
dev_contexts, dev_questions, dev_guids = read_dev_klue("/content/drive/MyDrive/dataset/test.json")

  0%|          | 0/3709 [00:00<?, ?it/s]

In [ ]:
# def read_dev_klue(path):
#     with open(path, 'rb') as f:
#         klue_dict = json.load(f)

#     contexts = []
#     questions = []
#     answers = []
#     guids = []

#     for group in tqdm(klue_dict['data']):
#         for passage in group['paragraphs']:
#             context = passage['context']
#             for qa in passage['qas']:
#                 question = qa['question']
#                 guid = qa['guid']
#                 #temp_answer = []
#                 #for answer in qa['answers']:
#                     #temp_answer.append(answer['text'])
#                 #if len(temp_answer) != 0: # answers의 길이가 0 == 답변할 수 없는 질문
#                     #contexts.append(context)
#                     #questions.append(question)
#                     #answers.append(temp_answer)
#                 contexts.append(context)##
#                 questions.append(question)##
#                 guids.append(guid)

#     #return contexts, questions, answers
#     return contexts, questions , guids

In [ ]:
# #dev_contexts, dev_questions, dev_answers = read_dev_klue("test.json")
# dev_contexts, dev_questions, dev_guids = read_dev_klue("/content/drive/MyDrive/dataset/test.json")

In [47]:
import re
def remove_post(text):
        ''' 불필요한 기호 제거 '''
        text = text.strip()
        text = re.sub("''", "'", text)
        text = re.sub('""', '"', text)
        text = re.sub("‘‘", "‘", text)
        text = re.sub("’’", "’", text)
        text = re.sub("  ", " ", text)
        text = re.sub("#", "", text)
        return text

In [48]:
# start, end logit의 확률값을 이용한 예측 정답값
# logit의 상위 5개 확률을 리스트로 뽑아 틀린 정답이었다면 다음 확률로 넘어가서 확인.

def to_list(tensor):
    return tensor.detach().cpu().tolist()

def logits_change(input_ids, STA_logits, END_logits):

    # 로짓의 확률값 ~ 상위 5개를 선택 
    # 틀린 추론이었다면 다음 선택 (틀린 추론 : start > end, 길이가 너무 긴 문장.)
    change_logit = 0
    cnt = 0
    
    # 기존 정답
    save_s = STA_logits
    save_e = END_logits

    STK_start_index, STK_end_index = save_s.argmax(dim=-1), save_e.argmax(dim=-1)
    save_pred_ids = tokenizer.decode(input_ids[0][STK_start_index: STK_end_index + 1])
    print(save_pred_ids) 

    # 바뀐 정답
    STA_logits = to_list(STA_logits)[0]
    END_logits = to_list(END_logits)[0]

    start_idx_and_logit = sorted(enumerate(STA_logits), key=lambda x: x[1], reverse=True)
    end_idx_and_logit = sorted(enumerate(END_logits), key=lambda x: x[1], reverse=True)

    start_idx_and_logit = start_idx_and_logit[:5]
    end_idx_and_logit = end_idx_and_logit[:5]

    TK_start_index, TK_end_index = start_idx_and_logit, end_idx_and_logit

    for i in range(5):
        if TK_start_index[i][0] > TK_end_index[i][0] or TK_end_index[i][0] - TK_start_index[i][0] > 10 : 
            cnt += 1
            continue
        else : 
            change_logit += 1
            pred_ids = input_ids[0][TK_start_index[i][0]: TK_end_index[i][0] + 1]
            pred_ids = tokenizer.decode(pred_ids)
            print(pred_ids, 'change')
            break

    if change_logit == 0 :
        return save_pred_ids
    elif cnt == 5:
        return ''
    else : 
        if pred_ids == save_pred_ids:
            print('same answer')
            return pred_ids
        else :
            print('different answer')
            return pred_ids

    

# start_logits , end_logits
# index를 추적하면서 시작, 종료 index에 대한 확률이 가장 높은것을 선택하는 방법.
# 만약에 차이가 큰 start, end 값을 반환할때 이 정보들을 저장하지 않고 넘긴다면? 


In [49]:
def prediction(contexts, questions, guids):
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    model.load_state_dict(torch.load('./output_model_best'))
    model.to(device)
    
    model.eval()
    
    result = []
    
    with torch.no_grad():
        
        for context, question, guid in zip(contexts, questions, guids):
            encodings = tokenizer(context, question, max_length=512, truncation=True,
                                     padding="max_length", return_token_type_ids=False)
            encodings = {key: torch.tensor([val]) for key, val in encodings.items()}
            
            input_ids = encodings["input_ids"].to(device)
            attention_mask = encodings["attention_mask"].to(device)
            
            outputs = model(input_ids, attention_mask=attention_mask)
            start_logits, end_logits = outputs.start_logits, outputs.end_logits

            ######
            pred =logits_change(input_ids, start_logits, end_logits)
            '''
            token_start_index, token_end_index = start_logits.argmax(dim=-1), end_logits.argmax(dim=-1)
            pred_ids = input_ids[0][token_start_index: token_end_index + 1]
            
            if token_start_index > token_end_index:
                pred = ''
            else:
                pred = tokenizer.decode(pred_ids)
                pred = pred[:10] # 1.5~1.8
            '''
            pred = pred[:10]
            pred = remove_post(pred)

            tp = (guid,pred)
            
            result.append(tp)
    return result

In [50]:
pred_answers = prediction(dev_contexts, dev_questions, dev_guids)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
2001년
2001년 change
same answer
한삼인분 ’
한삼인분 ’ change
same answer
1619년
1619년 change
same answer
취약계층과 고소득층
취약계층과 고소득층 change
same answer
은
은 change
same answer
0. 9 %
0. 9 % change
same answer
핑구스 PSI 2011
핑구스 PSI 2011 change
same answer
디지털 유통
디지털 유통 change
same answer
세인트앤드루스대
세인트앤드루스대 change
same answer
유틀란트 해전
유틀란트 해전 change
same answer
47, 746명
47, 746명 change
same answer
13 %
13 % change
same answer

겨울스포츠 change
different answer
덴마크 닐피스크
덴마크 닐피스크 change
same answer
박원순
박원순 change
same answer
댄스 가수
댄스 가수 change
same answer
파라과이
파라과이 change
same answer
1972
1972 change
same answer
녹색
녹색 change
same answer
전완부 근육
전완부 근육 change
same answer
로맨스
로맨스 change
same answer
1986년
1986년 change
same answer
서울옥션
서울옥션 change
same answer
29일
29일 change
same answer
인사동 아라아트센터
인사동 아라아트센터 change
same answer
내성
내성 change
same answer
수원 아주대병원
수원 아주대병원 change
same answer
2008
2008 change
same answer
12월 12일
12월 12일 change
same answer
티무르
티무르 change
same answer
7
7 ch

In [51]:
pred_answers

[('d14cb73158624cf094c546d856fd3c80', '뉴 530i 25주'),
 ('906631384e91493ebe1c7f34aea6f241', '독일 뒤셀도르프로'),
 ('35e61dcb479643448a2cb7d326ae50a6', '페이스북'),
 ('075e761b370040cb9041eecd39afc27c', '페이스북'),
 ('e67ed38f3dd944be94d5b4c53731f334', '마드리드'),
 ('80cacfdfe76442b0b39568e8b193d066', '국제 원자재값 하락'),
 ('78a80cca941c46379a5c65b2f1392f7c', '2조6000억원'),
 ('5c19b9781f8a4f0faa9045274c38a8b4', '운영허가 인증서'),
 ('d2844b7141cb4a2594d433e9c1e5cd0c', '그물 모양 구조'),
 ('be39b91f52a04b0aaafc96ff505cb2fc', '노르웨이'),
 ('98fe72c173f642e29f9284f1769fd4f6', '16'),
 ('1756f8a643124b52aa773a0f38b7b6fa', '필리핀'),
 ('1f1ef695105049079e093bae6f4ad8e7', '김교성'),
 ('acbc4be5f9d04b76a30451f8c6baf277', '79달러'),
 ('393657048abd435b9059b00ed54dcc47', '존 위클리프'),
 ('08ee8dae9f6744cf91d79e0782426d33', '존 위클리프'),
 ('0fd8aca8b9844e3d85e0379272600e3b', '지기스문트'),
 ('fa773f05fd224cbab0327d1459e38f8a', '복정역'),
 ('89810801a2f446ef9ea0b74cdc478f3c', '한울'),
 ('30a8f6b62fa142209b241d35e308002b', '서울대 음대'),
 ('4957b7070c3a463aaac97f0255ed

In [52]:
import csv
f = open('pred_practice.csv','w', newline='')
wr = csv.writer(f)
wr.writerow(['Id','Predicted'])

for tp in pred_answers:
    wr.writerow([tp[0],tp[1]])

f.close()

In [53]:
# pred_answers에서 도출한 정답 비교
import csv

def calculate_Leven(source, ref, result_file):
    with open(source, 'r') as input1:
        with open(ref, 'r') as input2:
            with open(result_file, 'w') as csvoutput:
                reader1 = csv.reader(input1)
                reader2 = list(csv.reader(input2))
                writer = csv.writer(csvoutput)
                result = []
                mean = []
                headers = next(reader1)
                result.append(headers)
                index = 0
                for row1 in reader1:
                    #print("First row")
                    #print(row1[1])
                    index+=1
                    #print(reader2[index][1])
                    a = levenshtein_distance(row1[1], reader2[index][1])
                    #print(row1[1],'////',reader2[index][1])
                    '''
                    max = 0
                    while max < 1:
                        for row2 in reader2:
                            a = distance(row1[1],row2[1])
                            print(a)
                            b = 1 - a/len(row1[1])
                            if b > max:
                                max = b
                                SKU = row2[1]
                    '''
                    mean.append(a)
                    row1.append(a)
                    result.append(row1)
                mean1 = sum(mean) / len(mean)
                print(mean1)
                writer.writerows(result)


In [54]:
calculate_Leven('/content/drive/MyDrive/dataset/klue.csv', 'pred_practice.csv', 'result_file.csv')

2.8295908183632736
